<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

This project is part of the Applied Data Science Capstone that explores and clusters the neighborhoods in Toronto, Canada. But first, a dataframe of the list of postal codes will be created. After that, the data of latitude and the longitude coordinates of each neighborhood will be added. The coordinates are tracked from Google Maps. Lastly, the clustering of neighborhoods in Toronto.

### 1. Create Dataframe

In [1]:
import numpy as np
import pandas as pd

#!conda install -c anaconda beautifulsoup4   #uncomment if installing for the first time

from bs4 import BeautifulSoup

Download list of postal codes of Canada using the URL

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table = pd.read_html(url)

In [3]:
neighborhoods = table[0]
neighborhoods

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Then, convert the downloaded list to dataframe

We only need to process the cells that have an assigned borough. Hence, remove data with Borough column equals to "Not assigned"

In [4]:
neighborhoods2 = neighborhoods[neighborhoods.Borough != 'Not assigned']
neighborhoods2

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


More than one neighborhood can exist in one postal code area. If exist, they needed to be combined into one row with the neighborhoods separated with a comma.
So, determine if there are duplicates in postal code.

In [5]:
duplicates = neighborhoods2[neighborhoods2.duplicated(['Postal Code'])]
print(duplicates)

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


As showed in the result above, there seem to be no duplicates in postal code.

Now, if a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough. The code below will do this.

In [6]:
neighborhoods2.loc[(neighborhoods2.Borough!="Not assigned") & (neighborhoods2.Neighbourhood=="Not assigned"), 'Neighbourhood'] = neighborhoods2.Borough
neighborhoods2

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Lastly, print the number of rows and columns of the final dataframe.

In [7]:
neighborhoods2.shape

(103, 3)

### Next is to add the latitude and the longitude coordinates of each neighborhood.

In [8]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

In [9]:
lat_lng = pd.read_csv('Geospatial_Coordinates.csv')
lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
lat_lng.shape

(103, 3)

In [11]:
neighborhoods3 = pd.merge(neighborhoods2, lat_lng)
neighborhoods3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
neighborhoods3.shape

(103, 5)

### 2. Explore and cluster the neighborhoods in Toronto

In [13]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes

import folium # map rendering library

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.11.0             |   py37h89c1867_0         168 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

boto3-1.17.20        | 70 KB     | ##################################### | 100% 
pysocks-1.7.1        | 27 KB     | ##################################### | 100% 
nss-3.62             | 2.1 MB    | ##################################### | 100% 
yarl-1.6.3           | 141 KB    | ##################################### | 100% 
lzo-2.10             | 314 KB    | ##################################### | 100% 
fontconfig-2.13.1    | 344 KB    | ##################################### | 100% 
async_generator-1.10 | 18 KB     | ##################################### | 100% 
python_abi-3.7       | 4 KB      | ##################################### | 100% 
secretstorage-3.3.1  | 24 KB     | ##################################### | 100% 
tensorflow-base-1.14 | 87.6 MB   | ##################################### | 100% 
gmp-6.2.1            | 806 KB    | ##################################### | 100% 
dask-core-2021.2.0   | 681 KB    | ##################################### | 100% 
libssh2-1.9.0        | 225 K

openssl-1.1.1j       | 2.1 MB    | ##################################### | 100% 
openjpeg-2.4.0       | 525 KB    | ##################################### | 100% 
ca-certificates-2020 | 137 KB    | ##################################### | 100% 
click-7.1.2          | 64 KB     | ##################################### | 100% 
pthread-stubs-0.4    | 5 KB      | ##################################### | 100% 
wcwidth-0.2.5        | 33 KB     | ##################################### | 100% 
tornado-6.1          | 646 KB    | ##################################### | 100% 
entrypoints-0.3      | 8 KB      | ##################################### | 100% 
psycopg2-2.8.6       | 168 KB    | ##################################### | 100% 
bzip2-1.0.8          | 484 KB    | ##################################### | 100% 
pyrsistent-0.17.3    | 89 KB     | ##################################### | 100% 
olefile-0.46         | 32 KB     | ##################################### | 100% 
_openmp_mutex-4.5    | 5 KB 

gst-plugins-base-1.1 | 2.5 MB    | ##################################### | 100% 
pyjwt-2.0.1          | 17 KB     | ##################################### | 100% 
fastcache-1.1.0      | 30 KB     | ##################################### | 100% 
pip-21.0.1           | 1.1 MB    | ##################################### | 100% 
mkl-service-2.3.0    | 54 KB     | ##################################### | 100% 
mkl-2020.4           | 215.6 MB  | ##################################### | 100% 
plotly-4.14.3        | 5.9 MB    | ##################################### | 100% 
nbformat-5.1.2       | 66 KB     | ##################################### | 100% 
gettext-0.19.8.1     | 3.6 MB    | ##################################### | 100% 
icu-68.1             | 13.0 MB   | ##################################### | 100% 
backcall-0.2.0       | 13 KB     | ##################################### | 100% 
mysql-libs-8.0.23    | 1.8 MB    | ##################################### | 100% 
sqlite-3.34.0        | 1.4 M

done


Select only boroughs that contain the word Toronto.

In [14]:
toronto_data = neighborhoods3[neighborhoods3['Borough'].str.contains('Toronto')]
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [23]:
toronto_data.shape

(39, 5)

Use geopy library to get the latitude and longitude values of Toronto. 
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [16]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim   # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0



geopy-2.1.0          | 64 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

In [17]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, utilize the Foursquare API to explore the neighborhoods and segment them. Let's explore the first neighborhood in the dataframe

In [24]:
toronto_data.loc[2, 'Neighbourhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [26]:
neighborhood_latitude = toronto_data.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[2, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Now, let's get the top 10 venues that are in Regent Park, Harbourfront within a radius of 500 meters.
But first, let's create the GET request URL. Name your URL url.

In [27]:
CLIENT_ID = '2YBS3MFZM5HAES4QBLOHFKW2M5PSRDULBEA2KG1DJTNDAYT1' #Foursquare ID
CLIENT_SECRET = 'OWOTCHSJY1UBF4USGBTIT5R4RINVGA5X55NT4BBKKAU0WSBY' #Foursquare Secret
VERSION = '20210304'

LIMIT = 10 #limit of number of venues returned by Foursquare API
radius = 500 #define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2YBS3MFZM5HAES4QBLOHFKW2M5PSRDULBEA2KG1DJTNDAYT1&client_secret=OWOTCHSJY1UBF4USGBTIT5R4RINVGA5X55NT4BBKKAU0WSBY&v=20210304&ll=43.6542599,-79.3606359&radius=500&limit=10'

Send the GET request and examine the results.

In [30]:
import requests #library to handle requests
from pandas.io.json import json_normalize #tranform JSON file into a pandas dataframe

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6040ad818675981deb99d247'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

Now, let's explore all the neighborhoods in Toronto within 500 meters.

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [37]:
# To see the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(350, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [38]:
# To check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,10,10,10,10,10,10
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",10,10,10,10,10,10
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
Christie,10,10,10,10,10,10
Church and Wellesley,10,10,10,10,10,10
"Commerce Court, Victoria Hotel",10,10,10,10,10,10
Davisville,10,10,10,10,10,10


In [39]:
# To determinte the number of unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 120 uniques categories.


#### Analyze each of the neighborhood.

In [41]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arts & Crafts Store,...,Sushi Restaurant,Swim School,Tailor Shop,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# To examine the new dataframe size

toronto_onehot.shape

(350, 120)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [43]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Sushi Restaurant,Swim School,Tailor Shop,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.1,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.1,0.1,0.1,0.2,0.1,0.1,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
5,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
6,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.1,0.00,0.0,0.0
7,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
8,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
9,Davisville North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0


In [44]:
# To determine the new size

toronto_grouped.shape

(39, 120)

Let's print each neighborhood along with the top 5 most common venues

In [45]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0                 Farmers Market   0.2
1                           Park   0.1
2  Vegetarian / Vegan Restaurant   0.1
3                   Cocktail Bar   0.1
4                   Concert Hall   0.1


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop   0.2
1      Italian Restaurant   0.1
2                     Gym   0.1
3          Breakfast Spot   0.1
4  Furniture / Home Store   0.1


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0         Garden Center   0.1
1         Auto Workshop   0.1
2  Fast Food Restaurant   0.1
3        Farmers Market   0.1
4            Comic Shop   0.1


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge   0.2
1      Harbor / Marina   0.1
2  Rental Car Locati

Let's put above results into a pandas dataframe and display the top 10 venues for each neighborhood.

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Museum,Seafood Restaurant,Restaurant,Concert Hall,Liquor Store,Cocktail Bar,Vegetarian / Vegan Restaurant,Park,Coffee Shop
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bar,Café,Italian Restaurant,Bakery,Gym,Breakfast Spot,Pet Store,Furniture / Home Store,Creperie
2,"Business reply mail Processing Centre, South C...",Garden Center,Fast Food Restaurant,Comic Shop,Pizza Place,Burrito Place,Brewery,Restaurant,Auto Workshop,Farmers Market,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Rental Car Location,Airport,Airport Food Court,Airport Gate,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Department Store
4,Central Bay Street,Coffee Shop,Park,Spa,Gastropub,Modern European Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Wine Bar,Creperie,Dance Studio


#### Now, let's cluster into 5 the neighborhoods of Toronto using k-means.

In [58]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 0, 0, 4, 2, 4, 4, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [61]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Chocolate Shop,Breakfast Spot,Restaurant,Spa,Bakery,Historic Site,Gym / Fitness Center,Distribution Center,Park,Coffee Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Yoga Studio,Sushi Restaurant,Distribution Center,Italian Restaurant,Creperie,Coffee Shop,Burrito Place,Burger Joint,Beer Bar,Park
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Plaza,Clothing Store,Comic Shop,Pizza Place,Burrito Place,Music Venue,Theater,Thai Restaurant,Flea Market
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Food Truck,Middle Eastern Restaurant,Cosmetics Shop,Creperie,Gym,Restaurant,Japanese Restaurant,Italian Restaurant,Wine Bar
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Pub,Health Food Store,Wine Bar,Distribution Center,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop


Finally, let's visualize the resulting clusters

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's examine the first cluster.

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,0,Coffee Shop,Food Truck,Middle Eastern Restaurant,Cosmetics Shop,Creperie,Gym,Restaurant,Japanese Restaurant,Italian Restaurant,Wine Bar
24,Downtown Toronto,0,Coffee Shop,Park,Spa,Gastropub,Modern European Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Wine Bar,Creperie,Dance Studio
41,East Toronto,0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Cosmetics Shop,Brewery,Yoga Studio,Food & Drink Shop,Diner,Coffee Shop,Comic Shop
43,West Toronto,0,Coffee Shop,Bar,Café,Italian Restaurant,Bakery,Gym,Breakfast Spot,Pet Store,Furniture / Home Store,Creperie
54,East Toronto,0,Coffee Shop,Ice Cream Shop,Fish Market,Bookstore,Bakery,Seafood Restaurant,Pet Store,Gay Bar,Concert Hall,Cosmetics Shop
75,West Toronto,0,Gift Shop,Dessert Shop,Restaurant,Coffee Shop,Movie Theater,Dog Run,Italian Restaurant,Eastern European Restaurant,Cuban Restaurant,Department Store
86,Central Toronto,0,Coffee Shop,Bank,American Restaurant,Restaurant,Pub,Supermarket,Sushi Restaurant,Liquor Store,Fried Chicken Joint,Cosmetics Shop
87,Downtown Toronto,0,Airport Lounge,Rental Car Location,Airport,Airport Food Court,Airport Gate,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Department Store
